# Расчет данных по нескольким гео

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()

# Общие параметры для заданий

Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять только в одном месте.

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по медиа, в нашем случае это ID ресурса Avito
mart_filter = 'crossMediaResourceId = 1028'

# Задаем типы пользования Интернетом
ut = {'Desktop&Mobile':[1,2,3], 'Desktop':[1], 'Mobile':[2,3]}

# Задаем необходимые гео
geo = {'Большая Москва':'cityPop = 1', 
       'Санкт-Петербург':'cityPop = 2', 
       '700+ без Мск и Спб':'cityPop = 3', 
       '100-700':'cityPop IN (4,5)', 
       '100+':'cityPop100 = 1', 
       '0+':'cityPop100 IN (1,2)'}

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer', 'adr', 'adrPer', 'awr', 'awrPer']

# Расчет для нескольких гео с разбивкой по Usetype

In [ ]:
%%time

# Посчитаем данные в циклах
tasks = []
print('Отправка задач ')

for utype, num in ut.items():
    for geo_name, logical in geo.items():
        usetype_filter = num
        project_name = f"{geo_name} & {utype}" # Задаем для отображения в DataFrame
        geo_filter = f"{logical}" # Задаем фильтр по географии

        # Формируем задание для API Cross Web в формате JSON
        task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                     demo_filter, mart_filter, slices, statistics)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['usetype'] = f"{utype}"
        tsk['geo'] = f"{geo_name}"
        tsk['task'] = mtask.send_audience_task(task_json)
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')
print('')
        
# Ждем выполнения
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')        
        
# Получаем результат
results = []
for t in tasks:
    tsk = t['task']
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])
    df_result['geo'] = t['geo']
    df_result['usetype'] = t['usetype']
    results.append(df_result)
df_result = pd.concat(results)
df_result        

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('several-geo'))
df_result.to_excel(writer, 'Report', index=False)
writer.save()